# Task 1: How would you sample from the final states in the statevector or tensor representations?

To sample from the final states in the statevector, I will do



1.  Normalize the statevector to ensure it represents valid probability amplitudes.
2.   Compute the probabilities of each basis state by taking the absolute value squared of the amplitude for that state.

3.  Determine the sampled state index.




In [17]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Define basic quantum gates
I = np.array([[1, 0], [0, 1]])  # Identity gate
X = np.array([[0, 1], [1, 0]])  # NOT gate (Pauli-X)
H = 1/np.sqrt(2) * np.array([[1, 1], [1, -1]])  # Hadamard gate

CNOT = np.array([[1, 0, 0, 0],                  # CNOT gate
                 [0, 1, 0, 0],
                 [0, 0, 0, 1],
                 [0, 0, 1, 0]])

def initialize_state(n_qubits):
    """Initialize quantum state to |0...0>"""
    state = np.zeros(2**n_qubits)
    state[0] = 1                                  # |0...0> state
    return state

def apply_single_qubit_gate(gate, target_qubit, n_qubits):
    """Apply a single qubit gate to a specific qubit"""

    result = np.array([[1]])                      # Staring with Identity matrix

    for i in range(n_qubits):
        if i == target_qubit:
            result = np.kron(result, gate)
        else:
            result = np.kron(result, I)

    return result

def apply_cnot(control, target, n_qubits):
    """Apply CNOT gate between control and target qubits"""
    if abs(control - target) != 1:
        raise ValueError("CNOT can only be applied to adjacent qubits in this implementation")

    # For simplicity, assume control qubit comes before target qubit
    first = min(control, target)


    result = np.array([[1]])

    # Add identity matrices before CNOT
    for i in range(first):
        result = np.kron(result, I)

    # Add CNOT
    result = np.kron(result, CNOT)

    # Add identity matrices after CNOT
    for i in range(first + 2, n_qubits):
        result = np.kron(result, I)

    return result

def simulate_circuit(n_qubits):
    """Simulate a quantum circuit with X, H, and CNOT gates"""
    # Initialize state
    state = initialize_state(n_qubits)

    # Apply X gate to first qubit
    x_matrix = apply_single_qubit_gate(X, 0, n_qubits)
    state = np.dot(x_matrix, state)

    # Apply H gate to second qubit
    h_matrix = apply_single_qubit_gate(H, 1, n_qubits)
    state = np.dot(h_matrix, state)

    # Apply CNOT between first two qubits
    if n_qubits >= 2:
        cnot_matrix = apply_cnot(0, 1, n_qubits)
        state = np.dot(cnot_matrix, state)

    return state

def sample_state(state, n_samples=1000):
    """Sample states from the final statevector"""
    # Normalize the statevector
    norm = np.linalg.norm(state)
    state = state / norm

    # Compute probability amplitudes
    probabilities = np.abs(state) ** 2

    # Sample n_samples times
    samples = np.random.choice(len(state), size=n_samples, p=probabilities)

    return samples

def index_to_state(index, n_qubits):
    """Convert an index to its corresponding qubit state representation"""
    return format(index, f'0{n_qubits}b')

# Simulate the circuit
n_qubits = 3
state = simulate_circuit(n_qubits)

# Sample states from the final statevector
n_samples = 1000
samples = sample_state(state, n_samples)

# Count occurrences of each state
unique, counts = np.unique(samples, return_counts=True)
for index, count in zip(unique, counts):
    qubit_state = index_to_state(index, n_qubits)
    probability = count / n_samples
    print(f"State |{qubit_state}>: {count} occurrences (probability: {probability:.4f})")

print(f"\nTotal samples: {n_samples}")

State |100>: 522 occurrences (probability: 0.5220)
State |110>: 478 occurrences (probability: 0.4780)

Total samples: 1000


For Tensor Representation

In [20]:
import numpy as np

# Define basic quantum gates
I = np.array([[1, 0], [0, 1]])  # Identity gate
X = np.array([[0, 1], [1, 0]])  # NOT gate (Pauli-X)
H = 1/np.sqrt(2) * np.array([[1, 1], [1, -1]])  # Hadamard gate

CNOT = np.array([[1, 0, 0, 0],                  # CNOT gate
                 [0, 1, 0, 0],
                 [0, 0, 0, 1],
                 [0, 0, 1, 0]])

def initialize_state(n_qubits):
    """Initialize the state tensor"""
    state = np.zeros((2,) * n_qubits)
    state[(0,) * n_qubits] = 1                  # Initialize the quantum tensor state
    return state

def apply_single_qubit_gate(gate, target_qubit, n_qubits):
    """Apply a single qubit gate to a specific qubit"""
    result = np.array([[1]])

    for i in range(n_qubits):
        if i == target_qubit:
            result = np.kron(result, gate)
        else:
            result = np.kron(result, I)

    return result

def apply_cnot(control, target, n_qubits):
    """Apply CNOT gate between control and target qubits"""
    if abs(control - target) != 1:
        raise ValueError("CNOT can only be applied to adjacent qubits in this implementation")

    result = np.array([[1]])

    for i in range(min(control, target)):
        result = np.kron(result, I)

    result = np.kron(result, CNOT)

    for i in range(min(control, target) + 2, n_qubits):
        result = np.kron(result, I)

    return result

def simulate_circuit(n_qubits):
    """Simulate a quantum circuit with X, H, and CNOT gates"""
    # Initialize state
    state = initialize_state(n_qubits)

    # Apply X gate to first qubit
    x_matrix = apply_single_qubit_gate(X, 0, n_qubits)
    state = np.tensordot(x_matrix, state, axes=0)

    # Apply H gate to second qubit
    h_matrix = apply_single_qubit_gate(H, 1, n_qubits)
    state = np.tensordot(h_matrix, state, axes=0)

    # Apply CNOT between first two qubits
    if n_qubits >= 2:
        cnot_matrix = apply_cnot(0, 1, n_qubits)
        state = np.tensordot(cnot_matrix, state, axes=0)

    return state

def sample_from_tensor(state, n_samples=100):
    """Sample states from the final tensor representation multiple times"""
    # Normalize the tensor state
    norm = np.linalg.norm(state)
    state = state / norm

    # Compute probabilities by summing over all dimensions
    probabilities = np.abs(state.flatten()) ** 2

    # Sample n_samples times from the probabilities
    samples = np.random.choice(len(probabilities), size=n_samples, p=probabilities)

    return samples

def index_to_state(index, n_qubits):
    """Convert an index to its corresponding qubit state representation"""
    return format(index, f'0{n_qubits}b')

# Simulate the circuit
n_qubits = 3
state = simulate_circuit(n_qubits)

# Sample states from the final tensor representation
n_samples = 100
samples = sample_from_tensor(state, n_samples)

# Count occurrences of each state
unique, counts = np.unique(samples, return_counts=True)
for index, count in zip(unique, counts):
    qubit_state = index_to_state(index, n_qubits)
    probability = count / n_samples
    print(f"State |{qubit_state}>: {count} occurrences (probability: {probability:.4f})")

print(f"\nTotal samples: {n_samples}")

State |100000>: 1 occurrences (probability: 0.0100)
State |10010110000>: 1 occurrences (probability: 0.0100)
State |10011111000>: 1 occurrences (probability: 0.0100)
State |1001111011000>: 1 occurrences (probability: 0.0100)
State |10010000100000>: 1 occurrences (probability: 0.0100)
State |11011100000000>: 1 occurrences (probability: 0.0100)
State |11011111011000>: 1 occurrences (probability: 0.0100)
State |101111100000000>: 1 occurrences (probability: 0.0100)
State |110100000100000>: 1 occurrences (probability: 0.0100)
State |110110000100000>: 1 occurrences (probability: 0.0100)
State |110110100000000>: 1 occurrences (probability: 0.0100)
State |1001000000000100000>: 1 occurrences (probability: 0.0100)
State |1001000000111011000>: 1 occurrences (probability: 0.0100)
State |1001000010111011000>: 1 occurrences (probability: 0.0100)
State |1001010010001101000>: 1 occurrences (probability: 0.0100)
State |1001010010011111000>: 1 occurrences (probability: 0.0100)
State |1001010010100000000

# Task 2: How about computing exact expectation values in the form <Ψ| Op |Ψ>?

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Define basic quantum gates
I = np.array([[1, 0], [0, 1]])  # Identity gate
X = np.array([[0, 1], [1, 0]])  # NOT gate (Pauli-X)
H = 1/np.sqrt(2) * np.array([[1, 1], [1, -1]])  # Hadamard gate

CNOT = np.array([[1, 0, 0, 0],                  # CNOT gate
                 [0, 1, 0, 0],
                 [0, 0, 0, 1],
                 [0, 0, 1, 0]])

def initialize_state(n_qubits):
    """Initialize quantum state to |0...0>"""
    state = np.zeros(2**n_qubits)
    state[0] = 1                                  # |0...0> state
    return state

def apply_single_qubit_gate(gate, target_qubit, n_qubits):
    """Apply a single qubit gate to a specific qubit"""

    result = np.array([[1]])                      # Starting with Identity matrix

    for i in range(n_qubits):
        if i == target_qubit:
            result = np.kron(result, gate)
        else:
            result = np.kron(result, I)

    return result

def apply_cnot(control, target, n_qubits):
    """Apply CNOT gate between control and target qubits"""
    if abs(control - target) != 1:
        raise ValueError("CNOT can only be applied to adjacent qubits in this implementation")

    # For simplicity, assume control qubit comes before target qubit
    first = min(control, target)

    result = np.array([[1]])

    # Add identity matrices before CNOT
    for i in range(first):
        result = np.kron(result, I)

    # Add CNOT
    result = np.kron(result, CNOT)

    # Add identity matrices after CNOT
    for i in range(first + 2, n_qubits):
        result = np.kron(result, I)

    return result

def simulate_circuit(n_qubits):
    """Simulate a quantum circuit with X, H, and CNOT gates"""
    # Initialize state
    state = initialize_state(n_qubits)

    # Apply X gate to first qubit
    x_matrix = apply_single_qubit_gate(X, 0, n_qubits)
    state = np.dot(x_matrix, state)

    # Apply H gate to second qubit
    h_matrix = apply_single_qubit_gate(H, 1, n_qubits)
    state = np.dot(h_matrix, state)

    # Apply CNOT between first two qubits
    if n_qubits >= 2:
        cnot_matrix = apply_cnot(0, 1, n_qubits)
        state = np.dot(cnot_matrix, state)

    return state

def compute_expectation_value(state, operator):
    """Compute the expectation value <Ψ| Op |Ψ>"""
    return np.vdot(state, np.dot(operator, state))


n_qubits = 3
state = simulate_circuit(n_qubits)

operator = apply_single_qubit_gate(H, 1, n_qubits)


expectation_value = compute_expectation_value(state, operator)
print(f"Expectation value: {expectation_value}")


Expectation value: 0.7071067811865474
